<a href="https://colab.research.google.com/github/francisco140472/TamanduateiEnorsig/blob/main/RIBEIRAO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pyodbc

In [ ]:
!pip install pymssql

In [ ]:
!pip install --upgrade -q PyDrive google-auth google-auth-oauthlib google-auth-httplib2

In [ ]:
# Instalar as bibliotecas necessárias
!pip install -q pymssql PyDrive

# Importar as bibliotecas
import pymssql
import pandas as pd
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import unicodedata
import re  # Para trabalhar com regex e remover zeros extras

# Autenticar no Google Colab e configurar o Google Drive
auth.authenticate_user()

gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Conectar ao banco SQL e obter os dados
server = '200.98.80.97'
database = 'CENSO_RIBERAO_PRETO'
username = 'sa'
password = 'SantoAndre2021'

try:
    # Conexão com o banco de dados
    conn = pymssql.connect(server, username, password, database)
    query = """
    SELECT
     [FUNCIONARIO],
     [RGL_MATRICULA],
     [ENDERECO]
   FROM [CENSO_RIBERAO_PRETO].[dbo].[vwAPP_CELULAR_PROG_FUNCIONARIOS]

    """
    df = pd.read_sql(query, conn)
    conn.close()

    # Verificação se o DataFrame está vazio
    if df.empty:
        print("⚠️ Nenhum dado foi retornado do banco de dados.")
    else:
        # Passo 1: Formatar todas as colunas
        def formatar_colunas(column):
            # 1️⃣ Transformar todos os valores para string
            column = column.astype(str)
            # 2️⃣ Remover acentos e caracteres especiais
            column = column.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
            # 3️⃣ Remover zeros que aparecem após caracteres especiais
            column = column.replace(r'[^a-zA-Z0-9]0+', '', regex=True)
            # 4️⃣ Remover espaços extras no início e fim
            column = column.str.strip()
            # 5️⃣ Converter tudo para maiúsculo (opcional)
            column = column.str.upper()
            return column

        # Aplicar a formatação para todas as colunas
        for coluna in df.columns:
            df[coluna] = formatar_colunas(df[coluna])

        print(f"📋 Dados formatados (5 primeiros registros):")
        print(df.head())

        # 🔥 Gerar HTML interativo com DataTables e corrigir layout
        def gerar_html_interativo(df, titulo="Tabela de Dados"):
            tabela_html = df.to_html(index=False, classes="display nowrap", escape=False)

            # Definir o alinhamento das colunas conforme seus nomes
            col_align = {
                '[FUNCIONARIO]': 'center',  # Exemplo: Alinhar para o centro
                ' [RGL_MATRICULA]': 'center',  # Alinhar para o centro
                'ENDERECO': 'left'  # Alinhar para a esquerda
            }

            # Criar a estrutura de alinhamento para as colunas
            column_defs = []
            for col in df.columns:
                align = col_align.get(col, 'left')  # Defina 'left' como valor padrão
                column_defs.append({"targets": df.columns.get_loc(col), "className": f"text-{align}"})

            html_template = f"""
            <!DOCTYPE html>
            <html lang="pt-br">
            <head>
                <meta charset="UTF-8">
                <meta name="viewport" content="width=device-width, initial-scale=1.0">
                <title>{titulo}</title>
                <link rel="stylesheet" type="text/css" href="https://cdn.datatables.net/1.10.21/css/jquery.dataTables.css">
                <style>
                    /* Estilos adicionais para centralizar e alinhar a tabela */
                    body {{
                        font-family: Arial, sans-serif;
                        margin: 20px;
                        text-align: center;
                    }}
                    .dataTables_wrapper {{
                        margin: 0 auto;
                        width: 90%;
                    }}
                    table {{
                        margin: auto;
                        border-collapse: collapse;
                    }}
                    table.dataTable {{
                        width: 100% !important;
                    }}
                </style>
            </head>
            <body>
                <h1>{titulo}</h1>
                <p>Digite o termo de busca nos campos abaixo para filtrar os dados.</p>
                {tabela_html}
                <script src="https://code.jquery.com/jquery-3.5.1.js"></script>
                <script src="https://cdn.datatables.net/1.10.21/js/jquery.dataTables.min.js"></script>
                <script>
                    $(document).ready(function() {{
                        $('table.display').DataTable({{
                            "scrollX": true,
                            "pageLength": 10,
                            "columnDefs": {column_defs},
                            "language": {{
                                "url": "//cdn.datatables.net/plug-ins/1.10.21/i18n/Portuguese-Brasil.json"
                            }}
                        }});
                    }});
                </script>
            </body>
            </html>
            """
            return html_template

        # 🔥 Gerar HTML completo e salvar em arquivo
        html_content = gerar_html_interativo(df, titulo="Tabela Completa Inativas")
        with open('index.html', 'w', encoding='utf-8') as f:
            f.write(html_content)

        # 🚀 Upload do arquivo HTML no Google Drive
        try:
            file_drive = drive.CreateFile({'title': 'index.html'})
            file_drive.SetContentFile('index.html')
            file_drive.Upload()
            print(f"✅ Acesse o arquivo HTML aqui: {file_drive['alternateLink']}")
        except Exception as e:
            print(f"❌ Erro ao fazer upload no Google Drive: {e}")

except Exception as e:
    print(f"❌ Erro ao conectar ou processar os dados: {e}")


In [ ]:
# Instalar as bibliotecas necessárias
!pip install -q pymssql PyDrive

# Importar as bibliotecas
import pymssql
import pandas as pd
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import unicodedata
import re  # Para trabalhar com regex e remover zeros extras

# Autenticar no Google Colab e configurar o Google Drive
auth.authenticate_user()

gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Conectar ao banco SQL e obter os dados
server = '200.98.80.97'
database = 'CENSO_RIBERAO_PRETO'
username = 'sa'
password = 'SantoAndre2021'

try:
    # Conexão com o banco de dados
    conn = pymssql.connect(server, username, password, database)
    query = """
    SELECT
     [FUNCIONARIO],
     [RGL_MATRICULA],
     [ENDERECO]
   FROM [CENSO_RIBERAO_PRETO].[dbo].[vwAPP_CELULAR_PROG_FUNCIONARIOS]

    """
    df = pd.read_sql(query, conn)
    conn.close()

    # Verificação se o DataFrame está vazio
    if df.empty:
        print("⚠️ Nenhum dado foi retornado do banco de dados.")
    else:
        # Passo 1: Formatar todas as colunas
        def formatar_colunas(column):
            # 1️⃣ Transformar todos os valores para string
            column = column.astype(str)
            # 2️⃣ Remover acentos e caracteres especiais
            column = column.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
            # 3️⃣ Remover zeros que aparecem após caracteres especiais
            column = column.replace(r'[^a-zA-Z0-9]0+', '', regex=True)
            # 4️⃣ Remover espaços extras no início e fim
            column = column.str.strip()
            # 5️⃣ Converter tudo para maiúsculo (opcional)
            column = column.str.upper()
            return column

        # Aplicar a formatação para todas as colunas
        for coluna in df.columns:
            df[coluna] = formatar_colunas(df[coluna])

        print(f"📋 Dados formatados (5 primeiros registros):")
        print(df.head())

        # 🔥 Gerar HTML com filtro suspenso por FUNCIONARIO
        def gerar_html_interativo_com_filtro(df, titulo="Tabela de Dados"):
            tabela_html = df.to_html(index=False, classes="display nowrap", escape=False)

            # Definir o alinhamento das colunas conforme seus nomes
            col_align = {
                '[FUNCIONARIO]': 'center',  # Exemplo: Alinhar para o centro
                ' [RGL_MATRICULA]': 'center',  # Alinhar para o centro
                'ENDERECO': 'left'  # Alinhar para a esquerda
            }

            # Criar a estrutura de alinhamento para as colunas
            column_defs = []
            for col in df.columns:
                align = col_align.get(col, 'left')  # Defina 'left' como valor padrão
                column_defs.append({"targets": df.columns.get_loc(col), "className": f"text-{align}"})

            html_template = f"""
            <!DOCTYPE html>
            <html lang="pt-br">
            <head>
                <meta charset="UTF-8">
                <meta name="viewport" content="width=device-width, initial-scale=1.0">
                <title>{titulo}</title>
                <link rel="stylesheet" type="text/css" href="https://cdn.datatables.net/1.10.21/css/jquery.dataTables.css">
                <style>
                    /* Estilos adicionais para centralizar e alinhar a tabela */
                    body {{
                        font-family: Arial, sans-serif;
                        margin: 20px;
                        text-align: center;
                    }}
                    .dataTables_wrapper {{
                        margin: 0 auto;
                        width: 90%;
                    }}
                    table {{
                        margin: auto;
                        border-collapse: collapse;
                    }}
                    table.dataTable {{
                        width: 100% !important;
                    }}
                    #filtroFuncionario {{
                        margin: 20px;
                        padding: 10px;
                    }}
                </style>
            </head>
            <body>
                <h1>{titulo}</h1>
                <p>Digite o termo de busca nos campos abaixo para filtrar os dados.</p>
                <div id="filtroFuncionario">
                    <label for="filtro">Filtrar por FUNCIONÁRIO: </label>
                    <input type="text" id="filtroFuncionarioInput" placeholder="Buscar por nome do funcionário" />
                </div>
                {tabela_html}
                <script src="https://code.jquery.com/jquery-3.5.1.js"></script>
                <script src="https://cdn.datatables.net/1.10.21/js/jquery.dataTables.min.js"></script>
                <script>
                    $(document).ready(function() {{
                        var table = $('table.display').DataTable({{
                            "scrollX": true,
                            "pageLength": 10,
                            "columnDefs": {column_defs},
                            "language": {{
                                "url": "//cdn.datatables.net/plug-ins/1.10.21/i18n/Portuguese-Brasil.json"
                            }}
                        }});

                        // Filtro para FUNCIONARIO
                        $('#filtroFuncionarioInput').on('keyup', function() {{
                            table.column(0).search(this.value).draw();  // Coluna 0 é a de FUNCIONARIO
                        }});
                    }});
                </script>
            </body>
            </html>
            """
            return html_template

        # 🔥 Gerar HTML completo com filtro e salvar em arquivo
        html_content = gerar_html_interativo_com_filtro(df, titulo="Tabela Completa Censo")
        with open('index.html', 'w', encoding='utf-8') as f:
            f.write(html_content)

        # 🚀 Upload do arquivo HTML no Google Drive
        try:
            file_drive = drive.CreateFile({'title': 'index.html'})
            file_drive.SetContentFile('index.html')
            file_drive.Upload()
            print(f"✅ Acesse o arquivo HTML aqui: {file_drive['alternateLink']}")
        except Exception as e:
            print(f"❌ Erro ao fazer upload no Google Drive: {e}")

except Exception as e:
    print(f"❌ Erro ao conectar ou processar os dados: {e}")


In [ ]:
# Instalar as bibliotecas necessárias
!pip install -q pymssql PyDrive

# Importar as bibliotecas
import pymssql
import pandas as pd
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Autenticar no Google Colab e configurar o Google Drive
auth.authenticate_user()

gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Configuração de conexão com o banco SQL
server = '200.98.80.97'
database = 'CENSO_RIBERAO_PRETO'
username = 'sa'
password = 'SantoAndre2021'

# Função para formatar colunas
def formatar_colunas(column):
    # Transformar valores em string
    column = column.astype(str)
    # Remover acentos e caracteres especiais
    column = column.str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
    # Remover zeros extras após caracteres especiais
    column = column.replace(r'[^a-zA-Z0-9]0+', '', regex=True)
    # Remover espaços extras e converter para maiúsculo
    column = column.str.strip().str.upper()
    return column

# Conectar ao banco SQL e executar a query com JOIN
try:
    conn = pymssql.connect(server, username, password, database)
    query = """

    SELECT
        FUNC.[FUNCIONARIO],
        FUNC.[RGL_MATRICULA],
        FUNC.[ENDERECO],
        EXE.[Geolocation],
        EXE.[AGENTE]
    FROM
        [CENSO_RIBERAO_PRETO].[dbo].[vwAPP_CELULAR_PROG_FUNCIONARIOS] FUNC
    INNER JOIN
        [CENSO_RIBERAO_PRETO].[dbo].[SYS_EXECUTADOS] EXE
    ON
        FUNC.[RGL_MATRICULA] = EXE.[RGL_MATRICULA]
    """
    df = pd.read_sql(query, conn)
    conn.close()

    # Verificar se o DataFrame está vazio
    if df.empty:
        print("⚠️ Nenhum dado foi retornado após o JOIN.")
    else:
        # Aplicar a formatação para todas as colunas
        for coluna in df.columns:
            df[coluna] = formatar_colunas(df[coluna])

        print(f"📋 Dados vinculados e formatados (5 primeiros registros):")
        print(df.head())

        # Função para gerar HTML interativo
        def gerar_html_interativo_com_filtro(df, titulo="Tabela de Dados Vinculados"):
            tabela_html = df.to_html(index=False, classes="display nowrap", escape=False)
            html_template = f"""
            <!DOCTYPE html>
            <html lang="pt-br">
            <head>
                <meta charset="UTF-8">
                <meta name="viewport" content="width=device-width, initial-scale=1.0">
                <title>{titulo}</title>
                <link rel="stylesheet" type="text/css" href="https://cdn.datatables.net/1.10.21/css/jquery.dataTables.css">
                <style>
                    body {{
                        font-family: Arial, sans-serif;
                        margin: 20px;
                    }}
                    .dataTables_wrapper {{
                        margin: 0 auto;
                        width: 90%;
                    }}
                </style>
            </head>
            <body>
                <h1>{titulo}</h1>
                {tabela_html}
                <script src="https://code.jquery.com/jquery-3.5.1.js"></script>
                <script src="https://cdn.datatables.net/1.10.21/js/jquery.dataTables.min.js"></script>
                <script>
                    $(document).ready(function() {{
                        $('table.display').DataTable({{
                            "scrollX": true,
                            "pageLength": 10,
                            "language": {{
                                "url": "//cdn.datatables.net/plug-ins/1.10.21/i18n/Portuguese-Brasil.json"
                            }}
                        }});
                    }});
                </script>
            </body>
            </html>
            """
            return html_template

        # Gerar HTML e salvar localmente
        html_content = gerar_html_interativo_com_filtro(df, titulo="Tabela Vinculada: Funcionários e Execuções")
        with open('index_vinculado.html', 'w', encoding='utf-8') as f:
            f.write(html_content)

        # Upload do HTML gerado para o Google Drive
        file_drive = drive.CreateFile({'title': 'index_vinculado.html'})
        file_drive.SetContentFile('index_vinculado.html')
        file_drive.Upload()
        print(f"✅ Acesse o arquivo HTML vinculado aqui: {file_drive['alternateLink']}")

except Exception as e:
    print(f"❌ Erro ao conectar ou processar os dados: {e}")
